In [1]:
!pip install resemblyzer

     |████████████████████████████████| 15.7MB 269kB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71357 sha256=463b350e58ea9c1742ef267f5d8afc66033ab8c6cb64fc32ef898e2753f472ff
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
  Created wheel for typing: filename=typing-3.7.4.3-cp36-none-any.whl size=26307 sha256=d57a5e8d1a602e4a18bec78983a5fcbc9f588c083130cb4f5978a654e31577e9
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built webrtcvad typing


In [1]:
from resemblyzer import VoiceEncoder, preprocess_wav
import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt
import librosa
import os
import sys
import pickle as pkl
import pyaudio
import wave
import struct

In [ ]:
encoder = VoiceEncoder()
reducer = pkl.load(open('reducer.sav','rb'))
knn = pkl.load(open('knn_model.sav','rb'))
projs = np.load('projections.npy')
labels = np.load('labels.npy')

In [ ]:
fs=16000
FRAMESIZE = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 10
NOFRAMES = int(RATE*RECORD_SECONDS // FRAMESIZE)
p = pyaudio.PyAudio()

In [ ]:
fig = plt.figure()
plt.ion()
ax = fig.add_subplot(111)
ax.scatter(projs[:,0],projs[:,1],c=labels)
plt.draw()

In [ ]:
val=True
while(val):
    print('recording')
    stream = p.open(format=FORMAT,channels=1,rate=RATE,input=True,frames_per_buffer=FRAMESIZE)
    data = stream.read(NOFRAMES*FRAMESIZE)
    decoded = np.frombuffer(data, dtype=np.int16)
    stream.stop_stream()
    stream.close()

    embed = encoder.embed_utterance(decoded/np.max(decoded))
    proj = reducer.transform([embed])
    print(knn.predict(proj))
    ax.scatter(proj[0][0],proj[0][1],color='black',marker='+')
    plt.draw()

    val = input("Continue? yes or no: ")
    if val.capitalize() != 'Yes': val = False

p.terminate()
plt.close()